In [ ]:
##Import data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#import data
nhl = pd.read_excel("./NHL data.xlsx")
nhl.info()

##Prepare data

#Eliminate rows in dataset with missing data
nhl=nhl.dropna()
nhl.info()

#add Goals per Game
nhl["G/GP"] = nhl["G"]/nhl["GP"]

#add Assist per Game
nhl["A/GP"] = nhl["A"]/nhl["GP"]

#add the year the players were born
nhl['BornInt'] = nhl['Born'].str.replace('\W', '', regex=True)
nhl=nhl.dropna()
nhl = nhl.astype({'BornInt':'int'})
conditions = [
    (nhl['BornInt'] < 10000),
    (nhl['BornInt'] >= 10000) & (nhl['BornInt'] < 100000),
    (nhl['BornInt'] >= 100000)]
choices = [2000,nhl['BornInt']//10000+2000, nhl['BornInt']//10000+1900]
nhl['YearBorn'] = np.select(conditions, choices)
del nhl['BornInt']

#add players age
nhl["age"] = 2022 - nhl['YearBorn']

#add BMI (Body Mass Index)
nhl = nhl.astype({'Ht':'int'})
nhl = nhl.astype({'Wt':'int'})
nhl["WtInKg"] = nhl["Wt"]*0.453592
nhl["HtInCm"] = nhl["Ht"]*2.54
nhl["HtInCm^2"] = nhl["HtInCm"]*nhl["HtInCm"]
nhl["BMI"] = nhl["Wt"]*nhl["HtInCm^2"]
del nhl['WtInKg']
del nhl['HtInCm']
del nhl['HtInCm^2']

#adding adjusted +/-
nhl["Adjusted +/-"] = nhl["+/-"]-nhl["E+/-"]

#adding penatlties (in minutes) per game
nhl["PIM/GP"] = nhl["PIM"]/nhl["GP"]

#adding unblocked shot attempts (Fenwick, USAT) taken by this individual per game
nhl["iFF/GP"] = nhl["iFF"]/nhl["GP"]

#adding shots on goal taken by this individual per game
nhl["iSF/GP"] = nhl["iSF"]/nhl["GP"]

#adding expected goals (weighted shots) for this individual per game
nhl["ixG/GP"] = nhl["ixG"]/nhl["GP"]

#adding all scoring chances taken by this individual per game
nhl["iSCF/GP"] = nhl["iSCF"]/nhl["GP"]

#adding an estimate of the player's setup passes (passes that result in a shot attempt) per game
nhl["Pass/GP"] = nhl["Pass"]/nhl["GP"]

#adding hits thrown by this individual per game
nhl["iHF/GP"] = nhl["iHF"]/nhl["GP"]

#adding hits taken by this individual per game
nhl["iHA/GP"] = nhl["iHA"]/nhl["GP"]

#adding individual shots taken that missed the net per game
nhl["iMiss/GP"] = nhl["iMiss"]/nhl["GP"]

#adding giveaways by this individual per game
nhl["iGVA/GP"] = nhl["iGVA"]/nhl["GP"]

#adding takeaways by this individual per game
nhl["iTKA/GP"] = nhl["iTKA"]/nhl["GP"]

#adding shots blocked by this individual per game
nhl["iBLK/GP"] = nhl["iBLK"]/nhl["GP"]

#adding penalties drawn by this individual per game
nhl["iPEND/GP"] = nhl["iPEND"]/nhl["GP"]

#adding penalties taken by this individual per game
nhl["iPENT/GP"] = nhl["iPENT"]/nhl["GP"]

#adding the team's scoring chances while this player was on the ice per game
nhl["SCF/GP"] = nhl["SCF"]/nhl["GP"]

#adding scoring chances allowed while this player was on the ice per game
nhl["SCA/GP"] = nhl["SCA"]/nhl["GP"]

#adding the team's goals while this player was on the ice
nhl["GF/GP"] = nhl["GF"]/nhl["GP"]

#adding goals allowed while this player was on the ice
nhl["GA/GP"] = nhl["GA"]/nhl["GP"]


#overview of the dataframe
nhl.info

##Prepare data for predicting the value of players

#selecting values of the dataset relevant to the prediction for the player's value
nhlSalaryPrediction = nhl[['Salary', 'age', 'BMI', 'Hand', 'Position', 'GP', 'G/GP', 'A/GP', '+/-', 'Adjusted +/-', 'PIM/GP', 'SH%', 'SV%', 'iFF/GP', 'iSF/GP', 'ixG/GP', 'iSCF/GP', 'Pass/GP', 'iHF/GP', 'iHA/GP', 'iMiss/GP', 'iGVA/GP', 'iTKA/GP', 'iBLK/GP', 'BLK%', 'FO%', '%FOT', 'iPENT/GP', 'iPEND/GP', 'SCF/GP', 'SCA/GP', 'GF/GP', 'GA/GP']]

#ckecking correlation between the selected stats for the valuation of the players
nhlSalaryPrediction.corr()

#checking wich stats have a correlation >0.7 (risk to distort the prediction model)
c = nhlSalaryPrediction.corr().abs()
s = c.unstack()
s = s[s<1]
s = s[s>0.7]
s.sort_values().drop_duplicates()

#Eliminate too strongly correlated stats to optimize the prediction model
del nhlSalaryPrediction['SCF/GP']
del nhlSalaryPrediction['SCA/GP']
del nhlSalaryPrediction['Adjusted +/-']
del nhlSalaryPrediction['iSF/GP']
del nhlSalaryPrediction['Pass/GP']
del nhlSalaryPrediction['GF/GP']
del nhlSalaryPrediction['ixG/GP']
del nhlSalaryPrediction['iBLK/GP']
del nhlSalaryPrediction['iPENT/GP']
del nhlSalaryPrediction['iFF/GP']

#Convert "Hand" and "Position" to dummy variable
nhlSalaryPrediction = pd.get_dummies(nhlSalaryPrediction, columns=['Hand'], drop_first=True)
nhlSalaryPrediction = pd.get_dummies(nhlSalaryPrediction, columns=['Position'], drop_first=True)

##RANDOM FOREST MODEL

#Create a variable for the logarithmized salary
nhlSalaryPrediction_log = nhlSalaryPrediction.copy()
nhlSalaryPrediction_log["logSalary"] = np.log(nhlSalaryPrediction["Salary"])
del nhlSalaryPrediction_log["Salary"]

#splitting dataset to training and test dataset
nhl_train_log, nhl_test_log = train_test_split(nhlSalaryPrediction_log, test_size=0.25, random_state=42)

#checking training dataset
nhl_train_log

# Create the random forest model
rf_log_model = RandomForestRegressor(n_estimators=100)

# Train the model using the training data
y = nhl_train_log["logSalary"]
x = nhl_train_log.loc[:, nhl_train_log.columns != 'logSalary']
rf_log_model.fit(x, y)

# Use the model to make predictions on the test data
y_test = nhl_test_log["logSalary"]
X_test = nhl_test_log.loc[:, nhl_test_log.columns != 'logSalary']
predictions_Salary_with_log = rf_log_model.predict(X_test)

#List of metric names
metric_names = ["R-squared", "MAE", "MSE"]

#List of metric functions
metric_functions = [metrics.r2_score, metrics.mean_absolute_error, metrics.mean_squared_error]


#Calculate RMSE
rmse_log_sal = sqrt(mean_squared_error(y_test, predictions_Salary_with_log))

# print the metrics         
for metric_name, metric_function in zip(metric_names, metric_functions):
  value = metric_function(y_test, predictions_Salary_with_log)
  print(f"{metric_name}: {value:.4f}")

# print the importance of each feature
importances = rf_log_model.feature_importances_
for i, importance in enumerate(importances):
    feature_name = nhl_train_log.columns[i]
    print("Feature %s: %f" % (feature_name, importance))
    
#Creating an Actual vs. Predicted Plot
def actual_vs_predicted_plot(y_test, prediction_Salary_test):
  min_value=np.array([y_test.min(), predictions_Salary_with_log.min()]).min()
  max_value=np.array([y_test.max(), predictions_Salary_with_log.max()]).max()
  fig, ax = plt.subplots(figsize=(10,5))
  ax.scatter(y_test,predictions_Salary_with_log, color="blue")
  ax.plot([min_value,max_value], [min_value, max_value], lw=4, color="green")
  ax.set_title("Actual vs Predicted Plot")
  ax.set_xlabel('Actual')
  ax.set_ylabel('Predicted')
  plt.show()

#Displaying Actual vs. Predicted Plot
actual_vs_predicted_plot(y_test, predictions_Salary_with_log)

#Predicting the salary for the entire dataset / for all players (RF with log salaries)

#creating variable for logarithm of salary
nhlFinalSalaryPrediction = nhlSalaryPrediction.copy()
nhlFinalSalaryPrediction["logSalary"] = np.log(nhlFinalSalaryPrediction["Salary"])
del nhlFinalSalaryPrediction["Salary"]

#Predicting salary (value of players)
y = nhlFinalSalaryPrediction["logSalary"]
X = nhlFinalSalaryPrediction.loc[:, nhlFinalSalaryPrediction.columns != 'logSalary']
predictions_Salary = rf_log_model.predict(X)
predictions_Salary_final = np.exp(predictions_Salary)
predictions_Salary_final

#adding Value of Players (predicted Salaries based on their stats) and the difference in valuation (overpaid or underpaid) to the main dataset
nhlFinal=nhl.copy()
nhlFinal["Player Value"] = predictions_Salary_final
nhlFinal["Difference in valuation"] = nhlFinal["Player Value"] - nhlFinal["Salary"]

#Overview of final dataset
nhlFinal

#Write nhlFinal dataframe to a CSV file
nhlFinal.to_csv("nhl_final.csv")


In [ ]:
from tkinter import *
from tkinter import ttk

players = []

# 1. open .csv file
# 2. read all lines and extract necessary information (salary and first + last name)
# 3. safe this information in an array
# 4. write a function to search in this array by player name and return salary, value and difference
# 5. make if fAncYY

# - salary: 1
# - last name: 13
# - first name: 14
# - value: 179
# - valuation Difference: 180

# player {
#   lastName
#   fistName
#   salary
#   value
#   valuationDifference
# }
def extractInfo(row):
    columns = row.split(',')

    if(len(columns) <= 1):
        return

    print(len(columns))

    lastName = columns[13].lower()
    firstName = columns[14].lower()
    salary = int(columns[1])
    value = int(float(columns[179]))
    valuationDifference = int(float(columns[180]))
    player = dict(lastName = lastName, firstName = firstName, salary = salary, value = value, valuationDifference = valuationDifference)
    players.append(player)

def findPlayer(firstName, lastName):
    firstName = firstName.lower()
    lastName = lastName.lower()
    for i in range(0, len(players)):
        player = players[i]
        if(player["firstName"] == firstName and player["lastName"] == lastName):
            return player
    return -1

f = open("./nhl_final.csv","r")
frows = f.readlines()

for i in range(1, len(frows)):
    extractInfo(frows[i])

f.close()

salaryLabel = None
valueLabel = None
valueDifferenceLabel = None

# Get rid of old data
def destroyLabel(label):
    if label is not None:
        label.destroy()

def getSalary():
    global salaryLabel
    global valueLabel
    global valueDifferenceLabel

    destroyLabel(salaryLabel)
    destroyLabel(valueLabel)
    destroyLabel(valueDifferenceLabel)

    inputText = entry.get().strip()
    splitName = inputText.split(" ") 
    player = findPlayer(splitName[0], splitName[1])

    salaryLabel = Label(win, text="Salary(in USD): " + str(player["salary"]), font= ('Century 15 bold')) 
    salaryLabel.pack(pady=20)

    valueLabel = Label(win, text="Value(in USD): " + str(player["value"]), font= ('Century 15 bold')) 
    valueLabel.pack(pady=20)

    valueDifferenceLabel = Label(win, text="Value Difference(in USD): " + str(player["valuationDifference"]), font= ('Century 15 bold')) 
    valueDifferenceLabel.pack(pady=20)

win= Tk()

#Create an Entry Widget
entry= ttk.Entry(win,font=('Century 12'),width=40)
entry.pack(pady= 30)

#Set the geometry of tkinter frame
win.geometry("750x350")

#Create a button to display the text of entry widget
button= ttk.Button(win, text="Enter", command= getSalary)
button.pack()

win.mainloop()